In [2]:
import numpy as np
import pandas as pd

# FACTORS

In [4]:
factors = pd.read_csv(
    './data/rus1000_stocks_factors.csv', 
    on_bad_lines='skip', 
    header = 2, 
    # nrows = 10000, 
    low_memory=False, 
    converters={'SEDOL': (lambda x: x[:6])},
    parse_dates=['DATE'], 
    index_col=[3, 2]
).groupby(
    ['Symbol', 'DATE']
).fillna(
    method='ffill'
).sort_index()

In [5]:
factors.head()

Company Name        FS_ID  RETURN  RCP  RBP   RSP   REP  \
SEDOL  DATE                                                                    
200001 2002-12-31  AMAZON.COM INC  RMCVZ9-S-US     9.0  6.0  1.0  28.0   8.0   
       2003-01-31  AMAZON.COM INC  RMCVZ9-S-US    97.0  6.0  1.0  35.0   7.0   
       2003-02-28  AMAZON.COM INC  RMCVZ9-S-US    68.0  6.0  1.0  27.0   8.0   
       2003-03-31  AMAZON.COM INC  RMCVZ9-S-US    98.0  6.0  1.0  27.0   9.0   
       2003-04-30  AMAZON.COM INC  RMCVZ9-S-US    61.0  6.0  1.0  21.0  10.0   

                    RDP  RPM71  RSTDEV  ...   BR1   BR2  EP1  EP2   RV1   RV2  \
SEDOL  DATE                             ...                                     
200001 2002-12-31  39.0   99.0    11.0  ...  82.0  97.0  9.0  6.0  83.0  84.0   
       2003-01-31  39.0   95.0    11.0  ...  93.0  97.0  9.0  6.0  88.0  41.0   
       2003-02-28  39.0   99.0    16.0  ...  93.0  92.0  9.0  7.0  82.0  86.0   
       2003-03-31  39.0   98.0    18.0  ...  87.0  71.0  8.0  6.0  25.0  47.0   
       2003-04-30  39.0   98.0    20.0  ...  93.0  88.0  7.0  5.0  50.0  22.0   

                      CTEF     9MFR     8MFR   LIT  
SEDOL  DATE                                         
200001 2002-12-31  60.1667  24.6395   6.1009  46.0  
       2003-01-31  55.6667  55.6667      NaN  46.0  
       2003-02-28  61.5000  14.5402  14.5381  48.0  
       2003-03-31  40.6667  32.2199  25.1065  48.0  
       2003-04-30  44.1667  11.4560  11.1843  50.0  

[5 rows x 26 columns]

In [6]:
factor_sedols = factors.index.get_level_values("SEDOL").unique()

# STOCK RETURNS

In [7]:
stock_returns = pd.read_csv(
    "./data/cleaned_return_data_sc.csv", 
    # nrows=3, 
    parse_dates=['DATE'], 
    index_col=0
).fillna(method='ffill').fillna(0)

In [8]:
return_sedols = stock_returns.columns

In [18]:
stock_returns.head()

,000124,000163,000191,000312,000371,000415,000424,000432,000445,000495,...,BZCRNM,BZCTKP,BZHJN8,BZHJN9,BZHJNF,BZHJNG,BZHJNJ,BZHJNL,BZHJNM,BZHJVR
DATE,,,,,,,,,,,,,,,,,,,,,
1995-01-01,-0.119,0.153,NaN,0.484,NaN,0.535,NaN,0.173,0.062,0.565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-02-01,3.014,-0.719,NaN,-0.196,NaN,0.562,NaN,-0.143,0.096,0.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-03-01,0.372,0.695,NaN,0.602,NaN,0.476,NaN,0.084,0.942,0.855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-04-01,-0.215,0.403,NaN,0.899,NaN,-0.112,NaN,0.461,-0.078,0.860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-05-01,-0.127,1.186,NaN,-0.962,NaN,0.127,NaN,-0.330,-0.101,-0.450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# FACTOR-RETURN CONSISTENCY

In [9]:
common_sedols = []
for s in return_sedols:
    if s in factor_sedols:
        common_sedols += [s]

In [ ]:
sedols = pd.Series(common_sedols, name="SEDOLS")
sedols.to_csv("data/sedols.csv", index=False)

In [13]:
factors.loc[(common_sedols[:300],), :].to_csv("data/rus1000_stocks_factors_subset.csv")

In [14]:
stock_returns[common_sedols].to_csv("data/cleaned_return_data.csv")

# BENCHMARK RETURNS

In [25]:
benchmark_returns = pd.read_csv(
    './data/Benchmark Returns.csv', 
    on_bad_lines='skip', 
    # nrows = 100, 
    low_memory=False, 
    parse_dates=['Date'], 
    index_col=[0]
)
benchmark_returns.index.name = "DATE"

In [26]:
benchmark_returns

,MSCI EM Bench Return,Russell 1000 Bench Return,MSCI ACWIXUS Bench Return
DATE,,,
2002-12-31,-0.033226,-0.056599,-0.032306
2003-01-31,-0.004354,-0.024234,-0.035107
2003-02-28,-0.026990,-0.015472,-0.020262
2003-03-31,-0.028355,0.010351,-0.019398
2003-04-30,0.089072,0.080728,0.096360
...,...,...,...
2019-04-30,0.021241,0.040384,0.027170
2019-05-31,-0.072246,-0.063724,-0.052596
2019-06-28,0.063223,0.070204,0.060711
